In [10]:
from scipy.integrate import odeint
import numpy as np
import matplotlib.pyplot as plt
from ExtendedKalmanFilter import EKF

In [11]:
initial_pos = np.array([0,0,0])
initial_speed = np.array([0,0])
measurements = np.array([
    [1,0,0],
    [1,0,90]
])

us = np.array([
    [1, 0],
    [0, 90]
])
ekf = EKF(1, initial_pos, initial_speed)
ekf.filter(measurements[0], us[0])
ekf.filter(measurements[1], us[1])

State Estimate Before EKF=[1.01  0.01  0.003]
Observation=[1 0 0]
State Estimate After EKF=[ 0.96809524 -0.03190476 -0.01952381]
State Estimate Before EKF=[ 9.78095238e-01 -2.19047619e-02  8.99834762e+01]
Observation=[ 1  0 90]
State Estimate After EKF=[ 9.49056604e-01 -5.09433962e-02  8.99693019e+01]


-------------------------------------------------------------PID--------------------------------------------------------------

In [12]:
# Initialisation variable
conversion = 1
conversion_pixel_mm = 10
integral = 0.0
previous_error = 0.0
decalage = 0
timer_period[0] = 100


#Initialisation valeur non importante dans le programme final
l_speed = 50
r_speed = 50

motor_left_target  = 0
motor_right_target = 0

ekf = EKF(1, initial_pos, initial_speed)

def calculate_directional_angle_2D(point1, point2, point3):
    vector1 = np.array(point1) - np.array(point2)
    vector3 = np.array(point3) - np.array(point2)

    # Calculate angles in radians
    angle1 = np.arctan2(vector1[1], vector1[0])
    angle3 = np.arctan2(vector3[1], vector3[0])

    # Calculate directional angle
    directional_angle = angle3 - angle1

    # Adjust the angle to be between -pi and pi
    directional_angle = (directional_angle + np.pi) % (2 * np.pi) - np.pi

    return directional_angle

# Changer les valeurs global,vitesse motteur,
def Reglage():
    global l_speed, r_speed, integral, previous_error,conversion,x_f,y_f,x_d,y_d,x_t,y_t

    # Déclaration des constantes PID
    kp = 1.0  # Terme proportionnel
    ki = 0.1  # Terme intégral
    kd = 0.01  # Terme dérivé
    tolerance = 0.01  # tolerance
    
    # Calculate the hypotenuse of the vector (point2 - point3)
    Hypothenus = np.linalg.norm(np.array([x_d,y_d]) - np.array([x_t,y_t]))

    # Calculate the directional angle between point1, point2, and point3
    Angle = calculate_directional_angle_2D([x_f,y_f],[x_d,y_d],[x_t,y_t])

    # Calculate decalage using sin(Angle) * Hypotenuse
    decalage = np.sin(Angle) * Hypothenus  

    if abs(decalage) > tolerance:
        # Calcul des termes PID
        P = kp * decalage
        I = ki * integral
        D = kd * (decalage - previous_error)

        # Calcul de la commande finale
        output = P + I + D
        
        l_speed = 600+(output * conversion)  # Assuming conversion is defined
        r_speed = 600-(output*conversion)
       
        # Mettre à jour les variables pour la prochaine itération
        previous_error = decalage
        integral += decalage


#Boucle principal 

@onevent
def timer0():
    global motor_left_target, motor_right_target, l_speed, r_speed  
    [x_t,y_t,teta]=ekf.filter(measurements[0], us[0])
    ekf.filter(measurements[1], us[1])
    Reglage()
    # Set motor powers
    motor_left_target = round(l_speed) 
    motor_right_target = round(r_speed) 
        


NameError: name 'timer_period' is not defined

In [17]:
x,y,teta=ekf.filter(measurements[0], us[0])

State Estimate Before EKF=[0.79940078 0.63528358 1.89306629]
Observation=[1 0 0]
State Estimate After EKF=[0.88011541 0.19939503 0.69836745]


TypeError: cannot unpack non-iterable NoneType object

In [5]:
ekf.filter(measurements[0], us[0])

State Estimate Before EKF=[-0.20992708  0.32719976 34.53294203]
Observation=[1 0 0]
State Estimate After EKF=[ 0.4942384   0.08183813 13.17624931]
